In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [8]:
def update_station_urls():
    # URL of the webpage
    url = 'https://www.ndbc.noaa.gov/to_station.shtml'
    
    # Send a request to the webpage
    response = requests.get(url)
    response.raise_for_status()  # Check that the request was successful
    
    # Parse the webpage content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all station ID links
    station_links = soup.find_all('a', href=True)
    
    # Extract station IDs from the links
    station_ids = [link['href'].split('=')[-1] for link in station_links if 'station=' in link['href']]
    
        
    df = pd.DataFrame(station_ids,columns=['stations'])
    
    df.to_csv(r'./buoy_stations.csv')

    return

In [5]:
def get_stations_list():
    stations = pd.read_csv(r'./buoy_stations.csv')
    stations = stations.drop('Unnamed: 0', axis = 1)
    return stations

In [7]:
def download_buoy_data(stations):
    
    timestr = time.strftime("%Y%m%d-%H%M%S")

    errs = []
    buoy_err=[]
    
    suffix = ".spec" # the file suffix can determine the table downloaded (.spec is wave data)
    
    
    for buoy_id in stations['stations'].values:
        
        # URL of the webpage containing the table
        url = "https://www.ndbc.noaa.gov/data/realtime2/"+buoy_id+suffix 
        
        try:
            # Read the table data into a pandas DataFrame
            df = pd.read_csv(url, delim_whitespace=True)
            
            # Display the DataFrame
            df = df.iloc[1:]
            
            # Save the DataFrame to a CSV file (optional)
            df.to_csv("buoy_data/buoy_"+buoy_id+"__"+timestr+".csv", index=False)
        except Exception as e:
            buoy_err.append(buoy_id)
            errs.append(e)

    
    errors = pd.DataFrame({'buoy_id':buoy_err,'error':errs})
    
    print("Complete")
    print("Errors: " + str(len(errs)))
    
    return errors
    

In [9]:
## execute block

update_station_urls()

stations = get_stations_list()

e = download_buoy_data(stations)
e # errors out with 1593 error'd buoys

       #YY  MM  DD  hh  mm WVHT  SwH  SwP  WWH  WWP  SwD   WWD   STEEPNESS  \
0      #yr  mo  dy  hr  mn    m    m  sec    m  sec    -  degT           -   
1     2024  05  18  17  10  1.2  1.1  6.7  0.5  3.7   NW   NNW       STEEP   
2     2024  05  18  16  40  1.3  1.2  7.1  0.4  3.8    N    NW       STEEP   
3     2024  05  18  16  10  1.2  1.1  5.3  0.5  3.4  NNW   NNW  VERY_STEEP   
4     2024  05  18  15  40  1.2  1.1  6.2  0.5  3.8   NW    NW       STEEP   
...    ...  ..  ..  ..  ..  ...  ...  ...  ...  ...  ...   ...         ...   
1619  2024  04  15  00  10  1.4  1.2  8.3  0.7  5.3  ESE   WNW     AVERAGE   
1620  2024  04  14  23  40  1.4  0.9  8.3  1.0  6.2   SE   WNW     AVERAGE   
1621  2024  04  14  23  10  1.4  1.0  8.3  1.0  6.2   SE    NW       STEEP   
1622  2024  04  14  22  40  1.4  1.0  9.1  0.9  6.2  ESE   WNW       STEEP   
1623  2024  04  14  22  10  1.4  1.1  7.7  0.9  6.2  ESE    NW       STEEP   

      APD   MWD  
0     sec  degT  
1     5.1   306  
2     5.3

       #YY  MM  DD  hh  mm WVHT SwH  SwP WWH  WWP SwD   WWD   STEEPNESS  APD  \
0      #yr  mo  dy  hr  mn    m   m  sec   m  sec   -  degT           -  sec   
1     2024  05  10  04  40  1.6  MM   MM  MM   MM  MM    MM  VERY_STEEP  4.8   
2     2024  05  10  04  10  1.5  MM   MM  MM   MM  MM    MM  VERY_STEEP  4.6   
3     2024  05  10  03  40  1.5  MM   MM  MM   MM  MM    MM  VERY_STEEP  4.8   
4     2024  05  10  03  10  1.5  MM   MM  MM   MM  MM    MM  VERY_STEEP  4.7   
...    ...  ..  ..  ..  ..  ...  ..  ...  ..  ...  ..   ...         ...  ...   
2161  2024  03  26  02  10  3.2  MM   MM  MM   MM  MM    MM         NaN  7.2   
2162  2024  03  26  01  40  3.0  MM   MM  MM   MM  MM    MM         NaN  7.0   
2163  2024  03  26  01  10  3.3  MM   MM  MM   MM  MM    MM         NaN  7.4   
2164  2024  03  26  00  40  3.3  MM   MM  MM   MM  MM    MM         NaN  7.4   
2165  2024  03  26  00  10  3.0  MM   MM  MM   MM  MM    MM         NaN  7.1   

       MWD  
0     degT  
1      191  


       #YY  MM  DD  hh  mm WVHT  SwH  SwP  WWH  WWP  SwD   WWD STEEPNESS  APD  \
0      #yr  mo  dy  hr  mn    m    m  sec    m  sec    -  degT         -  sec   
1     2024  05  18  17  10  1.3  1.1  7.1  0.7  5.6   SE    SE     STEEP  5.8   
2     2024  05  18  16  40  1.4  1.2  7.7  0.6  5.3  ESE    SE   AVERAGE  5.9   
3     2024  05  18  16  10  1.5  1.4  7.1  0.5  4.2   SE   ESE     STEEP  6.1   
4     2024  05  18  15  40  1.5  1.4  7.7  0.5  4.2  ESE    SE   AVERAGE  6.0   
...    ...  ..  ..  ..  ..  ...  ...  ...  ...  ...  ...   ...       ...  ...   
2178  2024  04  03  02  10  1.4  1.4  6.2  0.4  3.8  SSE   SSE     STEEP  5.8   
2179  2024  04  03  01  40  1.5  1.5  7.7  0.4  3.8   SE   SSE   AVERAGE  6.0   
2180  2024  04  03  01  10  1.5  1.5  7.1  0.4  3.6   SE     S     STEEP  5.9   
2181  2024  04  03  00  40  1.4  1.4  7.1  0.4  3.7   SE    SE     STEEP  5.8   
2182  2024  04  03  00  10  1.5  1.4  7.7  0.4  4.0   SE   SSE   AVERAGE  5.8   

       MWD  
0     degT  
1

       #YY  MM  DD  hh  mm WVHT  SwH  SwP  WWH  WWP SwD   WWD   STEEPNESS  \
0      #yr  mo  dy  hr  mn    m    m  sec    m  sec   -  degT           -   
1     2024  05  18  17  10  2.4  0.0   MM  2.4  7.7  MM    SE       STEEP   
2     2024  05  18  16  40  2.3  0.0   MM  2.3  7.1  MM   SSE       STEEP   
3     2024  05  18  16  10  2.5  0.0   MM  2.5  7.7  MM   SSE       STEEP   
4     2024  05  18  15  40  2.5  0.0   MM  2.5  7.7  MM    SE       STEEP   
...    ...  ..  ..  ..  ..  ...  ...  ...  ...  ...  ..   ...         ...   
2140  2024  04  03  02  10  1.8  0.0   MM  1.8  6.7  MM   SSE       STEEP   
2141  2024  04  03  01  40  2.0  0.0   MM  2.0  6.7  MM   SSE       STEEP   
2142  2024  04  03  01  10  1.7  0.0   MM  1.7  6.7  MM   SSE       STEEP   
2143  2024  04  03  00  40  1.8  0.0   MM  1.8  6.7  MM   SSE       STEEP   
2144  2024  04  03  00  10  1.9  0.0   MM  1.9  6.2  MM   SSE  VERY_STEEP   

      APD   MWD  
0     sec  degT  
1     5.9   142  
2     5.8   148  
3  

       #YY  MM  DD  hh  mm WVHT  SwH  SwP  WWH  WWP  SwD   WWD   STEEPNESS  \
0      #yr  mo  dy  hr  mn    m    m  sec    m  sec    -  degT           -   
1     2024  05  18  17  10  1.3  1.3  8.3  0.3  3.8  ENE     E     AVERAGE   
2     2024  05  18  16  40  1.1  1.1  8.3  0.3  4.0    E     E     AVERAGE   
3     2024  05  18  16  10  1.2  1.1  7.7  0.3  4.0  ENE     E     AVERAGE   
4     2024  05  18  15  40  1.1  1.1  7.7  0.3  3.7  ENE   ENE     AVERAGE   
...    ...  ..  ..  ..  ..  ...  ...  ...  ...  ...  ...   ...         ...   
2182  2024  04  03  02  10  1.7  1.0  7.1  1.4  6.2  ESE     E       STEEP   
2183  2024  04  03  01  40  1.7  1.3  6.2  1.1  5.3    E   ENE  VERY_STEEP   
2184  2024  04  03  01  10  1.6  1.4  6.2  0.8  4.0    E   ENE       STEEP   
2185  2024  04  03  00  40  1.6  1.1  6.7  1.2  5.3    E     E       STEEP   
2186  2024  04  03  00  10  1.7  1.2  6.7  1.1  4.5    E     E       STEEP   

      APD   MWD  
0     sec  degT  
1     6.3    77  
2     6.0

...

       #YY  MM  DD  hh  mm WVHT  SwH  SwP  WWH  WWP  SwD   WWD STEEPNESS  APD  \
0      #yr  mo  dy  hr  mn    m    m  sec    m  sec    -  degT         -  sec   
1     2024  05  18  17  20  0.6  0.6  6.2  0.1  2.2  NaN   NaN       NaN  5.9   
2     2024  05  18  16  20  0.6  0.6  6.7  0.0   MM  NaN   NaN       NaN  6.0   
3     2024  05  18  15  20  0.6  0.5  6.7  0.2  2.4  NaN   NaN       NaN  5.8   
4     2024  05  18  14  20  0.6  0.6  6.7  0.2  2.3  NaN   NaN       NaN  5.9   
...    ...  ..  ..  ..  ..  ...  ...  ...  ...  ...  ...   ...       ...  ...   
1079  2024  04  03  04  20  0.6  0.6  5.0  0.0   MM  NaN   NaN       NaN  6.6   
1080  2024  04  03  03  20  0.6  0.5  5.3  0.2  3.4  NaN   NaN       NaN  6.2   
1081  2024  04  03  02  20  0.6  0.6  4.8  0.1  3.6  NaN   NaN       NaN  6.3   
1082  2024  04  03  01  20  0.6  0.6  5.6  0.2  3.6  NaN   NaN       NaN  6.2   
1083  2024  04  03  00  20  0.6  0.6  4.5  0.3  3.2  NaN   NaN       NaN  6.1   

       MWD  
0     degT  
1

       #YY  MM  DD  hh  mm WVHT  SwH   SwP  WWH  WWP  SwD   WWD STEEPNESS  \
0      #yr  mo  dy  hr  mn    m    m   sec    m  sec    -  degT         -   
1     2024  05  18  17  26  2.0  1.1  11.1  1.6  9.1  WNW    NW   AVERAGE   
2     2024  05  18  16  56  1.8  1.1  20.0  1.4  9.9  SSW    NW   AVERAGE   
3     2024  05  18  16  26  1.8  1.1  10.5  1.4  9.9   NW    NW   AVERAGE   
4     2024  05  18  15  56  1.9  1.1  16.7  1.5  9.1  SSW    NW   AVERAGE   
...    ...  ..  ..  ..  ..  ...  ...   ...  ...  ...  ...   ...       ...   
2184  2024  04  03  02  26  1.6  1.2  14.3  1.1  9.9  WNW   WNW     SWELL   
2185  2024  04  03  01  56  1.5  1.0  11.8  1.1  9.1    W   WNW   AVERAGE   
2186  2024  04  03  01  26  1.5  1.1  13.3  1.1  9.9   NW   WNW     SWELL   
2187  2024  04  03  00  56  1.4  1.0  14.3  1.1  8.3  WNW    NW   AVERAGE   
2188  2024  04  03  00  26  1.6  1.2  14.3  1.1  9.9    W   WNW     SWELL   

      APD   MWD  
0     sec  degT  
1     8.3   326  
2     8.2   206  
3  

       #YY  MM  DD  hh  mm WVHT SwH  SwP WWH  WWP SwD   WWD STEEPNESS  APD  \
0      #yr  mo  dy  hr  mn    m   m  sec   m  sec   -  degT         -  sec   
1     2024  05  18  17  40  0.1  MM   MM  MM   MM  MM    MM       NaN   MM   
2     2024  05  18  17  30  0.1  MM   MM  MM   MM  MM    MM       NaN   MM   
3     2024  05  18  17  20  0.1  MM   MM  MM   MM  MM    MM       NaN   MM   
4     2024  05  18  17  10  0.1  MM   MM  MM   MM  MM    MM       NaN   MM   
...    ...  ..  ..  ..  ..  ...  ..  ...  ..  ...  ..   ...       ...  ...   
2891  2024  04  23  18  30  0.4  MM   MM  MM   MM  MM    MM       NaN   MM   
2892  2024  04  23  18  20  0.4  MM   MM  MM   MM  MM    MM       NaN   MM   
2893  2024  04  23  18  10  0.4  MM   MM  MM   MM  MM    MM       NaN   MM   
2894  2024  04  23  17  50  0.4  MM   MM  MM   MM  MM    MM       NaN   MM   
2895  2024  04  23  17  40  0.4  MM   MM  MM   MM  MM    MM       NaN   MM   

       MWD  
0     degT  
1      135  
2      296  
3      203 

Complete
Errors: 1593


,buoy_id,error
0,1801583,HTTP Error 404: Not Found
1,1801589,HTTP Error 404: Not Found
2,21413,HTTP Error 404: Not Found
3,21414,HTTP Error 404: Not Found
4,21415,HTTP Error 404: Not Found
...,...,...
1588,41060,HTTP Error 404: Not Found
1589,41061,HTTP Error 404: Not Found
1590,41NT0,HTTP Error 404: Not Found
1591,43010,HTTP Error 404: Not Found
